# Creating a new planner in nuPlan <a name="introduction"></a>


## Setup

In [23]:
# Useful imports
import asyncio
import os
import hydra
import logging
import nest_asyncio
from IPython.core.display import display, HTML
from bokeh.io import output_notebook

/tmp/ipykernel_49264/4095267831.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [24]:
nest_asyncio.apply()
output_notebook()
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

Loading BokehJS ...

# Simulating the planner <a name="simulation"></a>

In [25]:
%env NUPLAN_DATA_ROOT=../../../datasets/nuplan/dataset
%env NUPLAN_MAPS_ROOT=../../data/nuplan/maps
%env NUPLAN_EXP_ROOT=../../data/nuplan/exp
%env NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/

%pwd

env: NUPLAN_DATA_ROOT=../../../datasets/nuplan/dataset
env: NUPLAN_MAPS_ROOT=../../data/nuplan/maps
env: NUPLAN_EXP_ROOT=../../data/nuplan/exp
env: NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/


'/home/ehdykhne/nuplan-devkit/experiments'

In [26]:
import hydra
import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf

from nuplan.common.utils.s3_utils import is_s3_path
from nuplan.planning.script.builders.simulation_builder import build_simulations
from nuplan.planning.script.builders.simulation_callback_builder import (
    build_callbacks_worker,
    build_simulation_callbacks,
)
from nuplan.planning.script.utils import (
    run_runners,
    set_default_path,
    set_up_common_builder,
)
from nuplan.planning.simulation.planner.abstract_planner import AbstractPlanner

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# If set, use the env. variable to overwrite the default dataset and experiment paths
set_default_path()

# If set, use the env. variable to overwrite the Hydra config
CONFIG_PATH = os.getenv("NUPLAN_HYDRA_CONFIG_PATH", "config/simulation")

if os.environ.get("NUPLAN_HYDRA_CONFIG_PATH") is not None:
    CONFIG_PATH = os.path.join("../../../../", CONFIG_PATH)

if os.path.basename(CONFIG_PATH) != "simulation":
    CONFIG_PATH = os.path.join(CONFIG_PATH, "simulation")
CONFIG_NAME = "default_simulation"

## Prepare the simulation config

In [27]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(
    os.getenv("NUPLAN_TUTORIAL_PATH", ""), "../nuplan/planning/script"
)
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

# Create a temporary directory to store the simulation artifacts

scenario_types = [
    # "accelerating_at_traffic_light_with_lead",
    # "crossed_by_bike",
    # "crossed_by_vehicle",
    # "on_intersection",
    # "on_stopline_crosswalk",
    # "on_stopline_stop_sign",
    # "on_stopline_traffic_light",
    # "on_traffic_light_intersection",
    # "starting_protected_cross_turn",
    # "starting_protected_noncross_turn",
    # "starting_right_turn",
    # "starting_straight_stop_sign_intersection_traversal",
    # "starting_straight_traffic_light_intersection_traversal",
    # "starting_u_turn",
    # "starting_unprotected_cross_turn",
    # "starting_unprotected_noncross_turn",
    # "stationary_at_crosswalk",
    # "stationary_at_traffic_light_with_lead",
    # "stationary_at_traffic_light_without_lead",
    # "traversing_crosswalk",
    # "traversing_intersection",
    "traversing_traffic_light_intersection",
]
scenario_builder = "val"  # [nuplan (uses trainval), nuplan_mini, test, val, train_boston, train_pittsburgh, train_singapore]
DATASET_PARAMS = [
    f"scenario_builder={scenario_builder}",
    "scenario_filter=all_scenarios",  # [all_scenarios, val14_split]
    f"scenario_filter.scenario_types={scenario_types}",  # there are 70 scenario types in the trainingset and 58 in the validation set including "unknown" which make up the majority
    # "scenario_filter.ego_displacement_minimum_m=10",  # use scenarios where the ego vehicle moves at least 10m
    #    'scenario_filter.remove_invalid_goals=true',  # remove scenarios where the goal is not invalid
    #    'scenario_filter.ego_start_speed_threshold=5',  # Exclusive threshold that the ego's speed must rise above (meters per second) for scenario to be kept
    #    'scenario_filter.stop_speed_threshold=10',  # Inclusive threshold that the ego's speed must fall below (meters per second) for scenario to be kept:
    "scenario_filter.map_names=[us-pa-pittsburgh-hazelwood]",  # [sg-one-north, us-ma-boston, us-pa-pittsburgh-hazelwood, us-nv-las-vegas-strip]
    # "scenario_filter.num_scenarios_per_type=200",  # use 10 scenarios per scenario type
    "scenario_filter.scenario_tokens=['60e03a4199bd5fbd']",  # List of scenarios to include (token)
    #    'scenario_filter.log_names=["2021.08.24.18.07.48_veh-45_01504_01722"]', # specific scenrios to simulate
    "scenario_filter.limit_total_scenarios=0.05",  # use n total scenarios if int, or if float smaller than 1, use n as a fraction of total scenarios (changes sampling frequency, unchanged leaves the frequency at 20Hz)
]
ckpt_dir = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/urbandriver_checkpoint.ckpt"
# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
print(simulation_hydra_paths.config_name)
cfg = hydra.compose(
    config_name=simulation_hydra_paths.config_name,
    overrides=[
        "+simulation=closed_loop_multiagent",
        "model=urban_driver_open_loop_model",
        "planner=ml_planner",
        # f"planner.pdm_hybrid_planner.checkpoint_path={ckpt_dir}" ,
        "planner.ml_planner.model_config=${model}",
        f"planner.ml_planner.checkpoint_path={ckpt_dir}",
        "observation.model_config=${model}",
        f"observation.checkpoint_path={ckpt_dir}",
        "worker=sequential",
        "+occlusion=true",
        "+occlusion.manager_type=wedge",  # options: [range, shadow, wedge]
        "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
        *DATASET_PARAMS,
    ],
)

output_folder = cfg.output_dir

default_simulation


## Launch simulation (within the notebook)

In [28]:
from nuplan.planning.script.run_simulation import build_simulation_runners
from nuplan.common.actor_state.tracked_objects_types import (
    AGENT_TYPES,
    STATIC_OBJECT_TYPES,
    TrackedObjectType,
)

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
runners, common_builder, cfg = build_simulation_runners(cfg)
print(len(runners))

Global seed set to 0


2023-12-20 03:28:05,856 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:20}  Building MultiMainCallback...
2023-12-20 03:28:05,876 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:35}  Building MultiMainCallback: 4...DONE!
2023-12-20 03:28:06,031 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2023-12-20 03:28:06,031 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: Sequential
2023-12-20 03:28:06,031 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 1
Number of GPUs per node: 0
Number of threads across all nodes: 1
2023-12-20 03:28:06,031 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2023-12-20 03:28:06,032 INFO {/home/e

MissingMandatoryValue: Missing mandatory value: observation.model_config
    full_key: observation.model_config
    object_type=dict

In [ ]:
runner = runners[0]

In [ ]:
from nuplan.common.actor_state.agent import Agent
from nuplan.common.actor_state.oriented_box import OrientedBox
from nuplan.common.actor_state.scene_object import SceneObjectMetadata
from nuplan.common.actor_state.state_representation import StateSE2, StateVector2D

inserted_agent = Agent(
    tracked_object_type=TrackedObjectType.VEHICLE,
    oriented_box=OrientedBox(StateSE2(588006, 4475692, 3 * 3.14 / 2), 5, 2, 2),
    velocity=StateVector2D(0, 0),
    metadata=SceneObjectMetadata(1623707858950113, "inserted", -2, "inserted"),
    angular_velocity=0.0,
)

inserted_goal = StateSE2(664433, 3997200, -3.14 / 2)

In [ ]:
runner.simulation.callback.on_simulation_start(runner.simulation.setup)

# Initialize all simulations
runner._initialize()

while runner.simulation.is_simulation_running():
    if runner.simulation._time_controller.get_iteration().index == 40:
        runner.simulation._observations.add_agent_to_scene(
            inserted_agent, inserted_goal, iteration.time_point
        )

    # Execute specific callback
    runner.simulation.callback.on_step_start(runner.simulation.setup, runner.planner)

    # Perform step
    planner_input = runner._simulation.get_planner_input()

    # Execute specific callback
    runner._simulation.callback.on_planner_start(
        runner.simulation.setup, runner.planner
    )

    # Plan path based on all planner's inputs
    trajectory = runner.planner.compute_trajectory(planner_input)

    # Propagate simulation based on planner trajectory
    runner._simulation.callback.on_planner_end(
        runner.simulation.setup, runner.planner, trajectory
    )

    iteration = runner.simulation._time_controller.get_iteration()
    print(iteration)
    runner.simulation.propagate(trajectory)

    # Execute specific callback
    runner.simulation.callback.on_step_end(
        runner.simulation.setup, runner.planner, runner.simulation.history.last()
    )

runner.simulation.callback.on_simulation_end(
    runner.simulation.setup, runner.planner, runner.simulation.history
)

SimulationIteration(time_point=TimePoint(time_us=1629835531599647), index=0)
SimulationIteration(time_point=TimePoint(time_us=1629835531699630), index=1)
SimulationIteration(time_point=TimePoint(time_us=1629835531799606), index=2)
SimulationIteration(time_point=TimePoint(time_us=1629835531899580), index=3)
SimulationIteration(time_point=TimePoint(time_us=1629835531999558), index=4)
SimulationIteration(time_point=TimePoint(time_us=1629835532099540), index=5)
SimulationIteration(time_point=TimePoint(time_us=1629835532199531), index=6)
SimulationIteration(time_point=TimePoint(time_us=1629835532299528), index=7)
SimulationIteration(time_point=TimePoint(time_us=1629835532399538), index=8)
SimulationIteration(time_point=TimePoint(time_us=1629835532499556), index=9)
SimulationIteration(time_point=TimePoint(time_us=1629835532599591), index=10)
SimulationIteration(time_point=TimePoint(time_us=1629835532699635), index=11)
SimulationIteration(time_point=TimePoint(time_us=1629835532799685), index=

In [ ]:
from tutorials.utils.tutorial_utils import visualize_history

visualize_history(runner.simulation._history, runner.scenario, bokeh_port=5007)

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.3.3)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


## Prepare the nuBoard config

In [ ]:
# Location of path with all nuBoard configs
CONFIG_PATH = "../nuplan/planning/script/config/nuboard"
CONFIG_NAME = "default_nuboard"

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(
    config_name=CONFIG_NAME,
    overrides=[
        "scenario_builder=nuplan_mini",  # set the database (same as simulation) used to fetch data for visualization
        f"simulation_path={[output_folder]}",  # nuboard file path(s), if left empty the user can open the file inside nuBoard
    ],
)

## Launch nuBoard (open in new tab - recommended)

In [ ]:
from tutorials.utils.tutorial_utils import visualize_history

visualize_history(runner.simulation._history, runner.scenario, bokeh_port=5008)

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.3.3)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


2023-12-20 03:21:53,988 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-20 03:21:53,988 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-20 03:21:53,988 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Error creating dataset. Could not read schema from 'data/nuplan/exp/exp/simulation/open_loop_boxes/2023.12.08.20.13.45/log.txt'. Is this a 'parquet' file?: Could not open Parquet input source 'data/nuplan/exp/exp/simulation/open_loop_boxes/2023.12.08.20.13.45/log.txt': Parquet magic bytes not found in footer. Either the file is corrupted or this 

Rendering a scenario: 100%|██████████| 1/1 [00:00<00:00, 16.72it/s]
INFO:tornado.access:200 GET / (10.40.112.28) 1064.44ms


2023-12-20 03:21:55,045 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  200 GET / (10.40.112.28) 1064.44ms
2023-12-20 03:21:55,045 INFO {/home/ehdykhne/nuplan-devkit/tutorials/utils/tutorial_utils.py:267}  Done rendering!


INFO:tornado.access:101 GET /ws (10.40.112.28) 0.63ms
INFO:bokeh.server.views.ws:WebSocket connection opened


2023-12-20 03:21:55,836 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  101 GET /ws (10.40.112.28) 0.63ms
2023-12-20 03:21:55,838 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-20 03:21:55,839 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2023-12-20 03:21:55,839 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Error creating dataset. Could not read schema from 'data/nuplan/exp/exp/simulation/open_loop_boxes/2023.12.08.20.13.45/log.txt'. Is this a 'parquet' file?: Could not open Parquet input source 'dat

Rendering a scenario: 100%|██████████| 1/1 [00:00<00:00, 62.71it/s]
INFO:tornado.access:200 GET / (10.40.112.28) 552.37ms
INFO:bokeh.server.views.ws:ServerConnection created


2023-12-20 03:21:56,388 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  200 GET / (10.40.112.28) 552.37ms
2023-12-20 03:21:56,515 INFO {/home/ehdykhne/nuplan-devkit/tutorials/utils/tutorial_utils.py:267}  Done rendering!
2023-12-20 03:22:29,636 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:575}  Frame deferred: 89
2023-12-20 03:22:30,035 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:897}  Frame dropped 89
2023-12-20 03:22:36,802 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:575}  Frame deferred: 98
2023-12-20 03:22:36,871 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:575}  Frame deferred: 101
2023-12-20 03:22:37,039 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:897}  Frame dropped 101
2023-12-20 03:22:37,303 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/